In [1]:
import re
import json

import apache_beam as beam
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib

In [2]:
def ExtractFields(line):
    record = json.loads(line)
    new_record = {}
    new_record['created_at'] = record['created_at'] # core column
    new_record['lang'] = record['lang']
    new_record['id'] = record['id']
    new_record['user_id'] = record['user']['id']
    new_record['user_name'] = record['user']['name']
    new_record['user_screen_name'] = record['user']['screen_name']
    new_record['text'] = record['text']
    new_record['json'] = json.dumps(record) # non core column, will be useful for replaying data pipeline if needed
    return new_record

In [3]:
p = beam.Pipeline(InteractiveRunner())

In [5]:
jsonTweets = (p | 'ReadMyFile' >> beam.io.ReadFromText('gs://raw_data_dev/data/postgres/*.json'))

In [6]:
parsedJsonTweets = (jsonTweets | 'ExtractFields' >> beam.Map(ExtractFields))

In [7]:
data = ib.collect(parsedJsonTweets)